In [113]:
%pip install -q gurobipy
%pip install -q pandas
%pip install -q numpy
import math
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [114]:
df = pd.read_csv("student_data.csv")

#Extract the number of students from the dataframe
NUM_STUDENTS = len(df)

#Create a list of students from 0 to num_students-1 for indexing purposes
students = list(range(NUM_STUDENTS))

#compute number of teams from 0 to num_teams-1 for indexing purposes
NUM_TEAMS = len(students)//2

df.describe()

preferences_dict = {}
for i in range(NUM_TEAMS):
    preferences_dict[i] = [np.random.randint(1,20) for i in range(1)]

,Student,Preference 1,Preference 2
count,37.000000,37.000000,37.000000
mean,18.000000,15.594595,16.054054
std,10.824355,11.422345,11.217412
min,0.000000,1.000000,0.000000
25%,9.000000,5.000000,9.000000
50%,18.000000,13.000000,13.000000
75%,27.000000,26.000000,24.000000
max,36.000000,36.000000,36.000000


In [115]:
# Create preferences dictionary from dataframe
preferences = {}
for i in students:
    preferences[i] = df.iloc[i,1:6].tolist()
print(preferences)

{0: [1, 10], 1: [7, 2], 2: [19, 27], 3: [4, 24], 4: [36, 0], 5: [21, 33], 6: [8, 3], 7: [28, 16], 8: [11, 14], 9: [35, 16], 10: [2, 12], 11: [8, 22], 12: [10, 21], 13: [34, 28], 14: [3, 8], 15: [2, 17], 16: [7, 9], 17: [13, 1], 18: [26, 11], 19: [2, 35], 20: [34, 13], 21: [12, 5], 22: [32, 0], 23: [4, 32], 24: [3, 9], 25: [30, 5], 26: [18, 35], 27: [14, 2], 28: [11, 13], 29: [31, 33], 30: [25, 36], 31: [16, 10], 32: [22, 23], 33: [5, 10], 34: [13, 20], 35: [26, 9], 36: [4, 30]}


In [116]:
m = gp.Model("student-teams")

# Create variables
# variable x[i,j] is 1 if student i is in team j and 0 otherwise
# variable y[j] is 1 if team j has at least 2 students and 0 otherwise
# there are at most as many teams as number of students / 2
NUM_TEAMS = len(students)//2
x = m.addVars(students, range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="x")
y = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="y")

# We create an auxiliary variable s[j] that is the score of team j
# The score of a team is the sum of the comfort of each student in each category
# We then maximize the minimum of the scores of all teams
s = m.addVars(range(NUM_TEAMS), name="s")

# We add an auxiliary variable z that is the minimum of the scores of all teams
z = m.addVar(name="z")

#decision variable that indicates if student i is assigned to their first choice
z1 = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="z")
print(z)

#decision variable that indicates if student i is assigned to their second choice
z2 = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="z2")

#decision variable that indicates if student i is assigned to their third choice
z3 = m.addVars(range(NUM_TEAMS), vtype=gp.GRB.BINARY, name="z3")


<gurobi.Var *Awaiting Model Update*>


In [117]:
# Create linear objective
# The objective is to maximize z, the minimum score of any team, as well as to minimize the number of teams
# There is another part to this objective, that is currently not being used because the model below makes the total number of teams a constant (making the y variables redundant)
z_wt = 1
m.setObjective(gp.quicksum(s[j] for j in range(NUM_TEAMS)), gp.GRB.MAXIMIZE)

In [118]:
# Add constraints

# each student is in exactly one team
m.addConstrs(
    gp.quicksum(x[i,j] for j in range(len(students)//2)) == 1
    for i in students
)

# each team has at least 2 students
m.addConstrs(
    gp.quicksum(x[i,j] for i in students) >= 2 * y[j]
    for j in range(len(students)//2)
)

# each team has at most 3 students
m.addConstrs(
    gp.quicksum(x[i,j] for i in students) <= 3 * y[j]
    for j in range(len(students)//2)
)

# z is the minimum of the scores of all teams
BIG_M = 45
m.addConstrs(
    z <= s[j] + BIG_M * (1 - y[j])
    for j in range(len(students)//2))


m.addConstrs(
    s[j] == gp.quicksum([
        gp.quicksum([
            10*x[preferences[i][0],j]*x[i,j] #+ 5*x[preferences[i][0],j]*x[i,j]
        ])
        for i in students
    ])
    for j in range(len(students)//2)
)



# Add constraint that total number of teams is ceil(number of students / 3) --> this actually makes the y variables unnecessary, I think
m.addConstr(gp.quicksum([y[j] for j in range(NUM_TEAMS)]) == math.ceil(len(students) / 3))

# Make teams ordered by preferences score
m.addConstrs(s[j] <= s[j+1] for j in range(NUM_TEAMS-1))

# Save model
m.write('student-teams.lp')

In [119]:
# Analyze model
def analyze_model(m):
    # Check if the model is optimal
    if m.status == gp.GRB.Status.OPTIMAL:
        print("Optimal!")
    if m.status != gp.GRB.Status.OPTIMAL:
        print('Optimization was stopped with status %d' % m.status)
        # If infeasible, compute and display IIS
        if m.status == gp.GRB.Status.INFEASIBLE:
            m.computeIIS()
            m.write('team-formation.ilp')

def get_teams(m):
    teams = []
    for j in range(NUM_TEAMS):
        if m.getVarByName('y[' + str(j) + ']').x > 0.5:
            team = []
            for i in range(NUM_STUDENTS):
                if m.getVarByName('x[' + str(i) + ',' + str(j) + ']').x > 0.5:
                    team.append(i)
            teams.append(team)
    return teams


In [120]:
# Optimize and analyze the output
m.optimize()

get_teams(m)

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads



GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license